In [1]:
import openmm
from openmm import unit, app

import mdtraj as md

from openmmforcefields.generators import SystemGenerator

import numpy as np

# ARG dipeptide

In [15]:
# Load PDBs
protein_filename = "/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/arg_vacuum.pdb"
protein_pdbfile = open(protein_filename, 'r')
protein_pdb = app.PDBFile(protein_pdbfile)
protein_pdbfile.close()
protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
protein_topology = protein_md_topology.to_openmm()
protein_n_atoms = protein_md_topology.n_atoms


In [16]:
# Create a system generator
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
forcefield_kwargs = {'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 3 * unit.amus}
periodic_forcefield_kwargs = {'nonbondedMethod': app.PME}
nonperiodic_forcefield_kwargs = None
small_molecule_forcefields = 'gaff-2.11'
molecules = []
system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

In [17]:
# Solvate with truncated octahedron
ionic_strength = 0.15 * unit.molar
water_model = 'tip3p'
modeller = app.Modeller(protein_topology, protein_positions)
geompadding = 2.0 * unit.nanometers
maxSize = max(max((pos[i] for pos in protein_positions))-min((pos[i] for pos in protein_positions)) for i in range(3))
vectors = openmm.Vec3(1,0,0), openmm.Vec3(1/3,2*np.sqrt(2)/3,0), openmm.Vec3(-1/3,np.sqrt(2)/3,np.sqrt(6)/3)
boxVectors = [(maxSize+geompadding)*v for v in vectors]
modeller.addSolvent(system_generator.forcefield, model=water_model, boxVectors=boxVectors, ionicStrength=ionic_strength)
solvated_topology = modeller.getTopology()
solvated_positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
solvated_positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)

# Generate system
solvated_system = system_generator.create_system(solvated_topology)

In [18]:
# Fix naked charges
atom_map = {atom.index: atom.residue.name for atom in solvated_topology.atoms()}
force_dict = {i.__class__.__name__: i for i in solvated_system.getForces()}
if 'NonbondedForce' in [k for k in force_dict.keys()]:
    nb_force = force_dict['NonbondedForce']
    for idx in range(nb_force.getNumParticles()):
        if atom_map[idx] in ['HOH', 'WAT']: # Do not add naked charge fix to water hydrogens
            continue
        charge, sigma, epsilon = nb_force.getParticleParameters(idx)
        if sigma == 0*unit.nanometer:
            new_sigma = 0.06*unit.nanometer
            nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
            print(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")
        if epsilon == 0*unit.kilojoule_per_mole:
            new_epsilon = 0.0001*unit.kilojoule_per_mole
            nb_force.setParticleParameters(idx, charge, sigma, new_epsilon)
            print(f"Changed particle {idx}'s epsilon from {epsilon} to {new_epsilon}")
            if sigma == 1.0 * unit.nanometer: # in protein.ff14SB, hydroxyl hydrogens have sigma=1 and epsilon=0
                new_sigma = 0.1*unit.nanometer
                nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
                print(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")

In [19]:
with open("/data/chodera/zhangi/perses_benchmark/repex/38/0/equil/apo_solvated.cif", "w") as f:
    app.PDBxFile.writeFile(solvated_topology, solvated_positions, f, keepIds=True)

In [20]:
with open("/data/chodera/zhangi/perses_benchmark/repex/38/0/equil/apo_solvated.xml", "w") as f:
    xml = openmm.XmlSerializer.serialize(solvated_system)
    f.write(xml)

In [21]:
solvated_topology.getPeriodicBoxVectors()

Quantity(value=(Vec3(x=2.9625, y=0.0, z=0.0), Vec3(x=0.9874999999999999, y=2.793071785686863, z=0.0), Vec3(x=-0.9874999999999999, y=1.3965358928434315, z=2.418871120998388)), unit=nanometer)

In [22]:
solvated_topology.getNumAtoms()

1910

In [23]:
solvated_system.getForce(3).getCutoffDistance()

Quantity(value=1.0, unit=nanometer)

# ASP dipeptide

In [17]:
# Load PDBs
protein_filename = "/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/asp_vacuum.pdb"
protein_pdbfile = open(protein_filename, 'r')
protein_pdb = app.PDBFile(protein_pdbfile)
protein_pdbfile.close()
protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
protein_topology = protein_md_topology.to_openmm()
protein_n_atoms = protein_md_topology.n_atoms


In [18]:
# Create a system generator
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
forcefield_kwargs = {'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 3 * unit.amus}
periodic_forcefield_kwargs = {'nonbondedMethod': app.PME}
nonperiodic_forcefield_kwargs = None
small_molecule_forcefields = 'gaff-2.11'
molecules = []
system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

In [19]:
# Solvate with truncated octahedron
ionic_strength = 0.15 * unit.molar
water_model = 'tip3p'
modeller = app.Modeller(protein_topology, protein_positions)
geompadding = 2.0 * unit.nanometers
maxSize = max(max((pos[i] for pos in protein_positions))-min((pos[i] for pos in protein_positions)) for i in range(3))
vectors = openmm.Vec3(1,0,0), openmm.Vec3(1/3,2*np.sqrt(2)/3,0), openmm.Vec3(-1/3,np.sqrt(2)/3,np.sqrt(6)/3)
boxVectors = [(maxSize+geompadding)*v for v in vectors]
modeller.addSolvent(system_generator.forcefield, model=water_model, boxVectors=boxVectors, ionicStrength=ionic_strength)
solvated_topology = modeller.getTopology()
solvated_positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
solvated_positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)

# Generate system
solvated_system = system_generator.create_system(solvated_topology)

In [20]:
# Fix naked charges
atom_map = {atom.index: atom.residue.name for atom in solvated_topology.atoms()}
force_dict = {i.__class__.__name__: i for i in solvated_system.getForces()}
if 'NonbondedForce' in [k for k in force_dict.keys()]:
    nb_force = force_dict['NonbondedForce']
    for idx in range(nb_force.getNumParticles()):
        if atom_map[idx] in ['HOH', 'WAT']: # Do not add naked charge fix to water hydrogens
            continue
        charge, sigma, epsilon = nb_force.getParticleParameters(idx)
        if sigma == 0*unit.nanometer:
            new_sigma = 0.06*unit.nanometer
            nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
            print(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")
        if epsilon == 0*unit.kilojoule_per_mole:
            new_epsilon = 0.0001*unit.kilojoule_per_mole
            nb_force.setParticleParameters(idx, charge, sigma, new_epsilon)
            print(f"Changed particle {idx}'s epsilon from {epsilon} to {new_epsilon}")
            if sigma == 1.0 * unit.nanometer: # in protein.ff14SB, hydroxyl hydrogens have sigma=1 and epsilon=0
                new_sigma = 0.1*unit.nanometer
                nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
                print(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")

In [21]:
with open("/data/chodera/zhangi/perses_benchmark/repex/38/1/equil/apo_solvated.cif", "w") as f:
    app.PDBxFile.writeFile(solvated_topology, solvated_positions, f, keepIds=True)

In [22]:
with open("/data/chodera/zhangi/perses_benchmark/repex/38/1/equil/apo_solvated.xml", "w") as f:
    xml = openmm.XmlSerializer.serialize(solvated_system)
    f.write(xml)

In [23]:
solvated_topology.getPeriodicBoxVectors()

Quantity(value=(Vec3(x=2.8033, y=0.0, z=0.0), Vec3(x=0.9344333333333333, y=2.6429765862669985, z=0.0), Vec3(x=-0.9344333333333333, y=1.3214882931334992, z=2.288884865314694)), unit=nanometer)

In [24]:
solvated_topology.getNumAtoms()

1620

In [25]:
solvated_system.getForce(3).getCutoffDistance()

Quantity(value=1.0, unit=nanometer)

# ALA dipeptide

In [30]:
# Load PDBs
protein_filename = "/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/ala_vacuum.pdb"
protein_pdbfile = open(protein_filename, 'r')
protein_pdb = app.PDBFile(protein_pdbfile)
protein_pdbfile.close()
protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
protein_topology = protein_md_topology.to_openmm()
protein_n_atoms = protein_md_topology.n_atoms


In [31]:
# Create a system generator
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
forcefield_kwargs = {'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 3 * unit.amus}
periodic_forcefield_kwargs = {'nonbondedMethod': app.PME}
nonperiodic_forcefield_kwargs = None
small_molecule_forcefields = 'gaff-2.11'
molecules = []
system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

In [32]:
# Solvate with truncated octahedron
ionic_strength = 0.15 * unit.molar
water_model = 'tip3p'
modeller = app.Modeller(protein_topology, protein_positions)
geompadding = 2.0 * unit.nanometers
maxSize = max(max((pos[i] for pos in protein_positions))-min((pos[i] for pos in protein_positions)) for i in range(3))
vectors = openmm.Vec3(1,0,0), openmm.Vec3(1/3,2*np.sqrt(2)/3,0), openmm.Vec3(-1/3,np.sqrt(2)/3,np.sqrt(6)/3)
boxVectors = [(maxSize+geompadding)*v for v in vectors]
modeller.addSolvent(system_generator.forcefield, model=water_model, boxVectors=boxVectors, ionicStrength=ionic_strength)
solvated_topology = modeller.getTopology()
solvated_positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
solvated_positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)

# Generate system
solvated_system = system_generator.create_system(solvated_topology)

In [33]:
# Fix naked charges
atom_map = {atom.index: atom.residue.name for atom in solvated_topology.atoms()}
force_dict = {i.__class__.__name__: i for i in solvated_system.getForces()}
if 'NonbondedForce' in [k for k in force_dict.keys()]:
    nb_force = force_dict['NonbondedForce']
    for idx in range(nb_force.getNumParticles()):
        if atom_map[idx] in ['HOH', 'WAT']: # Do not add naked charge fix to water hydrogens
            continue
        charge, sigma, epsilon = nb_force.getParticleParameters(idx)
        if sigma == 0*unit.nanometer:
            new_sigma = 0.06*unit.nanometer
            nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
            print(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")
        if epsilon == 0*unit.kilojoule_per_mole:
            new_epsilon = 0.0001*unit.kilojoule_per_mole
            nb_force.setParticleParameters(idx, charge, sigma, new_epsilon)
            print(f"Changed particle {idx}'s epsilon from {epsilon} to {new_epsilon}")
            if sigma == 1.0 * unit.nanometer: # in protein.ff14SB, hydroxyl hydrogens have sigma=1 and epsilon=0
                new_sigma = 0.1*unit.nanometer
                nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
                print(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")

In [34]:
with open("/data/chodera/zhangi/perses_benchmark/repex/38/3/equil/apo_solvated.cif", "w") as f:
    app.PDBxFile.writeFile(solvated_topology, solvated_positions, f, keepIds=True)

In [35]:
with open("/data/chodera/zhangi/perses_benchmark/repex/38/3/equil/apo_solvated.xml", "w") as f:
    xml = openmm.XmlSerializer.serialize(solvated_system)
    f.write(xml)

In [36]:
solvated_topology.getPeriodicBoxVectors()

Quantity(value=(Vec3(x=2.7648, y=0.0, z=0.0), Vec3(x=0.9216, y=2.6066784381660892, z=0.0), Vec3(x=-0.9216, y=1.3033392190830446, z=2.2574497469489767)), unit=nanometer)

In [37]:
solvated_topology.getNumAtoms()

1551

In [38]:
solvated_system.getForce(3).getCutoffDistance()

Quantity(value=1.0, unit=nanometer)

# THR dipeptide

In [2]:
# Load PDBs
protein_filename = "/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/thr_vacuum.pdb"
protein_pdbfile = open(protein_filename, 'r')
protein_pdb = app.PDBFile(protein_pdbfile)
protein_pdbfile.close()
protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
protein_topology = protein_md_topology.to_openmm()
protein_n_atoms = protein_md_topology.n_atoms


In [3]:
# Create a system generator
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
forcefield_kwargs = {'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 3 * unit.amus}
periodic_forcefield_kwargs = {'nonbondedMethod': app.PME}
nonperiodic_forcefield_kwargs = None
small_molecule_forcefields = 'gaff-2.11'
molecules = []
system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

In [4]:
# Solvate with truncated octahedron
ionic_strength = 0.15 * unit.molar
water_model = 'tip3p'
modeller = app.Modeller(protein_topology, protein_positions)
geompadding = 2.0 * unit.nanometers
maxSize = max(max((pos[i] for pos in protein_positions))-min((pos[i] for pos in protein_positions)) for i in range(3))
vectors = openmm.Vec3(1,0,0), openmm.Vec3(1/3,2*np.sqrt(2)/3,0), openmm.Vec3(-1/3,np.sqrt(2)/3,np.sqrt(6)/3)
boxVectors = [(maxSize+geompadding)*v for v in vectors]
modeller.addSolvent(system_generator.forcefield, model=water_model, boxVectors=boxVectors, ionicStrength=ionic_strength)
solvated_topology = modeller.getTopology()
solvated_positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
solvated_positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)

# Generate system
solvated_system = system_generator.create_system(solvated_topology)

In [5]:
# Fix naked charges
atom_map = {atom.index: atom.residue.name for atom in solvated_topology.atoms()}
force_dict = {i.__class__.__name__: i for i in solvated_system.getForces()}
if 'NonbondedForce' in [k for k in force_dict.keys()]:
    nb_force = force_dict['NonbondedForce']
    for idx in range(nb_force.getNumParticles()):
        if atom_map[idx] in ['HOH', 'WAT']: # Do not add naked charge fix to water hydrogens
            continue
        charge, sigma, epsilon = nb_force.getParticleParameters(idx)
        if sigma == 0*unit.nanometer:
            new_sigma = 0.06*unit.nanometer
            nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
            print(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")
        if epsilon == 0*unit.kilojoule_per_mole:
            new_epsilon = 0.0001*unit.kilojoule_per_mole
            nb_force.setParticleParameters(idx, charge, sigma, new_epsilon)
            print(f"Changed particle {idx}'s epsilon from {epsilon} to {new_epsilon}")
            if sigma == 1.0 * unit.nanometer: # in protein.ff14SB, hydroxyl hydrogens have sigma=1 and epsilon=0
                new_sigma = 0.1*unit.nanometer
                nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
                print(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")

Changed particle 16's epsilon from 0.0 kJ/mol to 0.0001 kJ/mol
Changed particle 16's sigma from 1.0 nm to 0.1 nm


In [6]:
with open("/data/chodera/zhangi/perses_benchmark/repex/38/5/equil/apo_solvated.cif", "w") as f:
    app.PDBxFile.writeFile(solvated_topology, solvated_positions, f, keepIds=True)

In [7]:
with open("/data/chodera/zhangi/perses_benchmark/repex/38/5/equil/apo_solvated.xml", "w") as f:
    xml = openmm.XmlSerializer.serialize(solvated_system)
    f.write(xml)

In [8]:
solvated_topology.getPeriodicBoxVectors()

Quantity(value=(Vec3(x=2.7648, y=0.0, z=0.0), Vec3(x=0.9216, y=2.6066784381660892, z=0.0), Vec3(x=-0.9216, y=1.3033392190830446, z=2.2574497469489767)), unit=nanometer)

In [9]:
solvated_topology.getNumAtoms()

1561

In [10]:
solvated_system.getForce(3).getCutoffDistance()

Quantity(value=1.0, unit=nanometer)

# VAL dipeptide

In [11]:
# Load PDBs
protein_filename = "/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/val_vacuum.pdb"
protein_pdbfile = open(protein_filename, 'r')
protein_pdb = app.PDBFile(protein_pdbfile)
protein_pdbfile.close()
protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
protein_topology = protein_md_topology.to_openmm()
protein_n_atoms = protein_md_topology.n_atoms


In [12]:
# Create a system generator
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
forcefield_kwargs = {'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 3 * unit.amus}
periodic_forcefield_kwargs = {'nonbondedMethod': app.PME}
nonperiodic_forcefield_kwargs = None
small_molecule_forcefields = 'gaff-2.11'
molecules = []
system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

In [13]:
# Solvate with truncated octahedron
ionic_strength = 0.15 * unit.molar
water_model = 'tip3p'
modeller = app.Modeller(protein_topology, protein_positions)
geompadding = 2.0 * unit.nanometers
maxSize = max(max((pos[i] for pos in protein_positions))-min((pos[i] for pos in protein_positions)) for i in range(3))
vectors = openmm.Vec3(1,0,0), openmm.Vec3(1/3,2*np.sqrt(2)/3,0), openmm.Vec3(-1/3,np.sqrt(2)/3,np.sqrt(6)/3)
boxVectors = [(maxSize+geompadding)*v for v in vectors]
modeller.addSolvent(system_generator.forcefield, model=water_model, boxVectors=boxVectors, ionicStrength=ionic_strength)
solvated_topology = modeller.getTopology()
solvated_positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
solvated_positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)

# Generate system
solvated_system = system_generator.create_system(solvated_topology)

In [14]:
# Fix naked charges
atom_map = {atom.index: atom.residue.name for atom in solvated_topology.atoms()}
force_dict = {i.__class__.__name__: i for i in solvated_system.getForces()}
if 'NonbondedForce' in [k for k in force_dict.keys()]:
    nb_force = force_dict['NonbondedForce']
    for idx in range(nb_force.getNumParticles()):
        if atom_map[idx] in ['HOH', 'WAT']: # Do not add naked charge fix to water hydrogens
            continue
        charge, sigma, epsilon = nb_force.getParticleParameters(idx)
        if sigma == 0*unit.nanometer:
            new_sigma = 0.06*unit.nanometer
            nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
            print(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")
        if epsilon == 0*unit.kilojoule_per_mole:
            new_epsilon = 0.0001*unit.kilojoule_per_mole
            nb_force.setParticleParameters(idx, charge, sigma, new_epsilon)
            print(f"Changed particle {idx}'s epsilon from {epsilon} to {new_epsilon}")
            if sigma == 1.0 * unit.nanometer: # in protein.ff14SB, hydroxyl hydrogens have sigma=1 and epsilon=0
                new_sigma = 0.1*unit.nanometer
                nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
                print(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")

In [15]:
with open("/data/chodera/zhangi/perses_benchmark/repex/38/7/equil/apo_solvated.cif", "w") as f:
    app.PDBxFile.writeFile(solvated_topology, solvated_positions, f, keepIds=True)

In [16]:
with open("/data/chodera/zhangi/perses_benchmark/repex/38/7/equil/apo_solvated.xml", "w") as f:
    xml = openmm.XmlSerializer.serialize(solvated_system)
    f.write(xml)

In [17]:
solvated_topology.getPeriodicBoxVectors()

Quantity(value=(Vec3(x=2.7648, y=0.0, z=0.0), Vec3(x=0.9216, y=2.6066784381660892, z=0.0), Vec3(x=-0.9216, y=1.3033392190830446, z=2.2574497469489767)), unit=nanometer)

In [18]:
solvated_topology.getNumAtoms()

1557

In [19]:
solvated_system.getForce(3).getCutoffDistance()

Quantity(value=1.0, unit=nanometer)

In [4]:
import pickle

In [5]:
with open("/data/chodera/zhangi/perses_benchmark/repex/38/2/2_apo.pickle", "rb") as f:
    htf = pickle.load(f)

INFO:rdkit:Enabling RDKit 2021.09.4 jupyter extensions


In [6]:
for atom in htf.hybrid_topology.atoms:
    print(atom, htf.hybrid_system.getParticleMass(atom.index))

ACE1-H1 4.0 Da
ACE1-CH3 3.0340000000000007 Da
ACE1-H2 4.0 Da
ACE1-H3 4.0 Da
ACE1-C 12.01 Da
ACE1-O 16.0 Da
ARG2-N 11.018 Da
ARG2-H 4.0 Da
ARG2-CA 9.018 Da
ARG2-HA 4.0 Da
ARG2-CB 4.530000000000001 Da
ARG2-HB2 4.0 Da
ARG2-HB3 4.0 Da
ARG2-C 12.01 Da
ARG2-O 16.0 Da
ARG2-CG 6.026000000000001 Da
ARG2-HG2 4.0 Da
ARG2-HG3 4.0 Da
ARG2-CD 6.026000000000001 Da
ARG2-HD2 4.0 Da
ARG2-HD3 4.0 Da
ARG2-NE 11.018 Da
ARG2-HE 4.0 Da
ARG2-CZ 12.01 Da
ARG2-NH1 8.026 Da
ARG2-HH11 4.0 Da
ARG2-HH12 4.0 Da
ARG2-NH2 8.026 Da
ARG2-HH21 4.0 Da
ARG2-HH22 4.0 Da
ARG2-HB2 4.0 Da
ARG2-HB3 4.0 Da
ARG2-HB1 4.0 Da
NME3-N 11.018 Da
NME3-H 4.0 Da
NME3-C 3.0340000000000007 Da
NME3-H1 4.0 Da
NME3-H2 4.0 Da
NME3-H3 4.0 Da
HOH4-O 15.99943 Da
HOH4-H1 1.007947 Da
HOH4-H2 1.007947 Da
HOH5-O 15.99943 Da
HOH5-H1 1.007947 Da
HOH5-H2 1.007947 Da
HOH6-O 15.99943 Da
HOH6-H1 1.007947 Da
HOH6-H2 1.007947 Da
HOH7-O 15.99943 Da
HOH7-H1 1.007947 Da
HOH7-H2 1.007947 Da
HOH8-O 15.99943 Da
HOH8-H1 1.007947 Da
HOH8-H2 1.007947 Da
HOH9-O 15.9994

In [9]:
with open("/data/chodera/zhangi/perses_benchmark/repex/38/0/equil/apo_solvated.xml", "r") as f:
    system = openmm.XmlSerializer.deserialize(f.read())

In [10]:
for i in range(system.getNumParticles()):
    print(system.getParticleMass(i))

3.0 Da
6.034 Da
3.0 Da
3.0 Da
12.01 Da
16.0 Da
12.018 Da
3.0 Da
10.018 Da
3.0 Da
8.026 Da
3.0 Da
3.0 Da
12.01 Da
16.0 Da
8.026 Da
3.0 Da
3.0 Da
8.026 Da
3.0 Da
3.0 Da
12.018 Da
3.0 Da
12.01 Da
10.026 Da
3.0 Da
3.0 Da
10.026 Da
3.0 Da
3.0 Da
12.018 Da
3.0 Da
6.034 Da
3.0 Da
3.0 Da
3.0 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.007947 Da
15.99943 Da
1.007947 Da
1.00

# LYS dipeptide

In [11]:
# Load PDBs
protein_filename = "/home/zhangi/choderalab/perses_benchmark/perses_protein_mutations/input/lys_vacuum.pdb"
protein_pdbfile = open(protein_filename, 'r')
protein_pdb = app.PDBFile(protein_pdbfile)
protein_pdbfile.close()
protein_positions, protein_topology, protein_md_topology = protein_pdb.positions, protein_pdb.topology, md.Topology.from_openmm(protein_pdb.topology)
protein_topology = protein_md_topology.to_openmm()
protein_n_atoms = protein_md_topology.n_atoms


In [12]:
# Create a system generator
forcefield_files = ['amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = openmm.MonteCarloBarostat(1.0 * unit.atmosphere, 300 * unit.kelvin, 50)
forcefield_kwargs = {'removeCMMotion': False, 'ewaldErrorTolerance': 0.00025, 'constraints' : app.HBonds, 'hydrogenMass' : 3 * unit.amus}
periodic_forcefield_kwargs = {'nonbondedMethod': app.PME}
nonperiodic_forcefield_kwargs = None
small_molecule_forcefields = 'gaff-2.11'
molecules = []
system_generator = SystemGenerator(forcefields=forcefield_files,
                                                barostat=barostat,
                                                forcefield_kwargs=forcefield_kwargs,
                                                periodic_forcefield_kwargs=periodic_forcefield_kwargs,
                                                nonperiodic_forcefield_kwargs=nonperiodic_forcefield_kwargs,
                                                small_molecule_forcefield=small_molecule_forcefields,
                                                molecules=molecules,
                                                cache=None)

DEBUG:openmmforcefields.system_generators:Trying GAFFTemplateGenerator to load gaff-2.11


In [13]:
# Solvate with truncated octahedron
ionic_strength = 0.15 * unit.molar
water_model = 'tip3p'
modeller = app.Modeller(protein_topology, protein_positions)
geompadding = 2.0 * unit.nanometers
maxSize = max(max((pos[i] for pos in protein_positions))-min((pos[i] for pos in protein_positions)) for i in range(3))
vectors = openmm.Vec3(1,0,0), openmm.Vec3(1/3,2*np.sqrt(2)/3,0), openmm.Vec3(-1/3,np.sqrt(2)/3,np.sqrt(6)/3)
boxVectors = [(maxSize+geompadding)*v for v in vectors]
modeller.addSolvent(system_generator.forcefield, model=water_model, boxVectors=boxVectors, ionicStrength=ionic_strength)
solvated_topology = modeller.getTopology()
solvated_positions = modeller.getPositions()

# Canonicalize the solvated positions: turn tuples into np.array
solvated_positions = unit.quantity.Quantity(value=np.array([list(atom_pos) for atom_pos in solvated_positions.value_in_unit_system(unit.md_unit_system)]), unit=unit.nanometers)

# Generate system
solvated_system = system_generator.create_system(solvated_topology)

In [14]:
# Fix naked charges
atom_map = {atom.index: atom.residue.name for atom in solvated_topology.atoms()}
force_dict = {i.__class__.__name__: i for i in solvated_system.getForces()}
if 'NonbondedForce' in [k for k in force_dict.keys()]:
    nb_force = force_dict['NonbondedForce']
    for idx in range(nb_force.getNumParticles()):
        if atom_map[idx] in ['HOH', 'WAT']: # Do not add naked charge fix to water hydrogens
            continue
        charge, sigma, epsilon = nb_force.getParticleParameters(idx)
        if sigma == 0*unit.nanometer:
            new_sigma = 0.06*unit.nanometer
            nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
            print(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")
        if epsilon == 0*unit.kilojoule_per_mole:
            new_epsilon = 0.0001*unit.kilojoule_per_mole
            nb_force.setParticleParameters(idx, charge, sigma, new_epsilon)
            print(f"Changed particle {idx}'s epsilon from {epsilon} to {new_epsilon}")
            if sigma == 1.0 * unit.nanometer: # in protein.ff14SB, hydroxyl hydrogens have sigma=1 and epsilon=0
                new_sigma = 0.1*unit.nanometer
                nb_force.setParticleParameters(idx, charge, new_sigma, epsilon)
                print(f"Changed particle {idx}'s sigma from {sigma} to {new_sigma}")

In [17]:
with open("/data/chodera/zhangi/perses_benchmark/repex/38/35/equil/apo_solvated.cif", "w") as f:
    app.PDBxFile.writeFile(solvated_topology, solvated_positions, f, keepIds=True)

In [18]:
with open("/data/chodera/zhangi/perses_benchmark/repex/38/35/equil/apo_solvated.xml", "w") as f:
    xml = openmm.XmlSerializer.serialize(solvated_system)
    f.write(xml)

In [19]:
solvated_topology.getPeriodicBoxVectors()

Quantity(value=(Vec3(x=2.9643, y=0.0, z=0.0), Vec3(x=0.9881, y=2.794768841961711, z=0.0), Vec3(x=-0.9881, y=1.3973844209808555, z=2.420340814844058)), unit=nanometer)

In [20]:
solvated_topology.getNumAtoms()

1938

In [21]:
solvated_system.getForce(3).getCutoffDistance()

Quantity(value=1.0, unit=nanometer)